In [24]:
import requests
import pandas as pd
import statistics as st
import numpy as np

In [2]:
# import path for imports
import sys
import os
import_path = os.path.join('c:', os.sep, "Users", "shane", "OneDrive", "MyDocuments", "AppDev", "my_imports")
sys.path.insert(1, import_path)

In [3]:
# import token
from config_keys import ncdc

In [5]:
# get state data from NOAA NCDC
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/locations"
token = ncdc
params = {
    'locationcategoryid' : 'ST',
    'startdate' : '2000-01-01',
    'enddate' : '2010-01-01',
    'limit' : 51
}

results = requests.get(base_url, params=params, headers={'token':token})

In [7]:
weather_states = results.json()['results']

In [45]:
def get_value(state, measure, month, year):
    """function to return an average value for a measurement"""
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"
    
    params = {
    'datasetid' : 'GSOM',
    'locationid' : state,
    'units' : 'standard',
    'datatypeid' : measure
    }
    
    # we need a 2 digit month
    if month < 10:
        params['startdate'] = f'{year}-0{month}-01'
        params['enddate'] = f'{year}-0{month}-01'
    else:
        params['startdate'] = f'{year}-{month}-01'
        params['enddate'] = f'{year}-{month}-01'
        
    #print(params)
    try:
        results = requests.get(base_url, params=params, headers={'token':token})
    
    # exception list from https://www.secopshub.com/t/handling-api-errors-using-python-requests/589
    except requests.exceptions.HTTPError as errh:
        print("An Http Error occurred:" + repr(errh))
        return(False)
    except requests.exceptions.ConnectionError as errc:
        print("An Error Connecting to the API occurred:" + repr(errc))
        return(False)
    except requests.exceptions.Timeout as errt:
        print("A Timeout Error occurred:" + repr(errt))
        return(False)
    except requests.exceptions.RequestException as err:
        print("An Unknown Error occurred" + repr(err))
        return(False)
    
    try:
        json_result = results.json()
    except:
        print("No JSON data")
        return(False)
    
    try:
        response_results = json_result['results']
        return(False)
    except KeyError:
        print('No results in json')
        return(False)
        
    measures = []
    for result in response_results:
        try:
            measures.append(result['value'])
        except KeyError:
            print("no value was found in results")
            return(False)

    return(st.mean(measures))
        

In [1]:
# create dictionary of lists for the datframe
# Get avergage monthly temperature data
# TAVG measure from GSOM is the 
weather_data = {
    'Year' : [],
    'Month' : [],
    'State' : [],
    'TAVG' : []
}

measurements = ['TAVG']
start_year = 2000
end_year = 2010

# build a list to capture the failures to try again if needed
faliures = []

for state in weather_states:
    print(state['id'])
#     for year in range(start_year, end_year + 1):
#         for month in range(1, 13):
#             for measure in measurements:
#                 print(f'{state["id"]} - {state["name"]} - {month} - {year} - {measure}')
                
#                 value = get_value(state["id"], measure, month, year)
                
#                 if value:
#                     weather_data[measure].append(value)
#                     weather_data['State'].append(state["name"])
#                     weather_data['Year'].append(year)
#                     weather_data['Month'].append(month)
#                 else:
#                     failures.append([state["id"], measure, month, year])


NameError: name 'weather_states' is not defined

In [37]:
weather_df = pd.DataFrame(weather_data)
weather_df

,Year,Month,State,TAVG
0,2000,1,Alabama,47.156
1,2000,2,Alabama,52.332
2,2000,3,Alabama,58.888
3,2000,4,Alabama,60.844
4,2000,5,Alabama,74.008
...,...,...,...,...
6727,2010,8,Wyoming,66.400
6728,2010,9,Wyoming,57.408
6729,2010,10,Wyoming,48.600
6730,2010,11,Wyoming,29.116


In [2]:
# check and see if we had any failures
faliures

[]

In [41]:
# write the data to a csv
weather_df.to_csv('Resources\temperatures.csv', index=False)